# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?

This supervised machine learning problem belongs to classification, because the outcome should be two categorical values: need early intervention or not. And we need make decisions whether a student might need early intervention based on the features.

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [2]:
# Import libraries
import numpy as np
import pandas as pd

In [3]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [4]:
student_data.head(3)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,passed
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,no,no,4,3,4,1,1,3,6,no
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,no,5,3,3,1,1,3,4,no
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,no,4,3,2,2,3,3,10,yes


In [5]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
n_students = student_data.shape[0]
n_features = student_data.shape[1] - 1
n_passed = sum(student_data.passed=='yes')
n_failed = sum(student_data.passed=='no')
grad_rate = 100.0*n_passed/(n_passed + n_failed)
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 30
Graduation rate of the class: 67.09%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [6]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [7]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [8]:
X_all.head()

,school_GP,school_MS,sex_F,sex_M,age,address_R,address_U,famsize_GT3,famsize_LE3,Pstatus_A,...,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences
0,1,0,1,0,18,0,1,1,0,1,...,1,0,0,4,3,4,1,1,3,6
1,1,0,1,0,17,0,1,1,0,0,...,1,1,0,5,3,3,1,1,3,4
2,1,0,1,0,15,0,1,0,1,0,...,1,1,0,4,3,2,2,3,3,10
3,1,0,1,0,15,0,1,1,0,0,...,1,1,1,3,2,2,1,1,5,2
4,1,0,1,0,16,0,1,1,0,0,...,1,0,0,4,3,2,1,2,5,4


In [9]:
X_all.shape

(395, 48)

In [53]:
# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
np.random.seed(1)
indices = np.random.permutation(num_all)
train_index = indices[:num_train]
test_index = indices[:-num_train]
X_train = X_all.loc[train_index]
y_train = y_all.loc[train_index]
X_test = X_all.loc[test_index]
y_test = y_all.loc[test_index]
print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data

Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

In [92]:
X_all.describe()

,school_GP,school_MS,sex_F,sex_M,age,address_R,address_U,famsize_GT3,famsize_LE3,Pstatus_A,...,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences
count,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,...,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000
mean,0.883544,0.116456,0.526582,0.473418,16.696203,0.222785,0.777215,0.711392,0.288608,0.103797,...,0.949367,0.832911,0.334177,3.944304,3.235443,3.108861,1.481013,2.291139,3.554430,5.708861
std,0.321177,0.321177,0.499926,0.499926,1.276043,0.416643,0.416643,0.453690,0.453690,0.305384,...,0.219525,0.373528,0.472300,0.896659,0.998862,1.113278,0.890741,1.287897,1.390303,8.003096
min,0.000000,0.000000,0.000000,0.000000,15.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,16.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,0.000000,4.000000,3.000000,2.000000,1.000000,1.000000,3.000000,0.000000
50%,1.000000,0.000000,1.000000,0.000000,17.000000,0.000000,1.000000,1.000000,0.000000,0.000000,...,1.000000,1.000000,0.000000,4.000000,3.000000,3.000000,1.000000,2.000000,4.000000,4.000000
75%,1.000000,0.000000,1.000000,1.000000,18.000000,0.000000,1.000000,1.000000,1.000000,0.000000,...,1.000000,1.000000,1.000000,5.000000,4.000000,4.000000,2.000000,3.000000,5.000000,8.000000
max,1.000000,1.000000,1.000000,1.000000,22.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,75.000000


There are 395 students and 30 features in the original dataset, after splitting and preprocessing, there are 300 and 95 students in training and testing datasets respectively, all of which have 48 features. The number of students are more than that of the features, so no high dimention problem. Now the features are all numerical without any missing values. Actually the feature values are all integers, some are even binary. Besides, the data is not skewed in terms of outcome, almost 2 thirds of students passed, and the rest failed.

First, we use a **Decision Tree Model** which are commonly used in operational research, classification and regresssion for decision making.
- Prons: It has good expressiveness, easy to understand a decision tree model and explain how each feature influences the calssifications through the nodes and branches. It is also compuationally cheap to use, missing values OK, can deal with irrelevant features. 
- Cons: for certain complex situations, i.e., if there are too many nodes to split on training data, overfitting may happen. 

In this case, the dataset contains many nominal features such as 'sex', 'address' and numeric features like 'age', 'absences', decison tree models can handle both types. There are not a large number of observations, just 300, which means it is easy to calculate gini or entropy for each feature and select proper nodes. Also it can work fast on prediction by just traversing the tree. Furthermore, we can explain which features are significant from the tree. 

In [72]:
# Train a model
import time

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Done!\nTraining time (secs): {:.3f}".format(end - start)

# TODO: Choose a model, import it and instantiate an object
import sklearn.tree
clf = sklearn.tree.DecisionTreeClassifier(random_state=1)#Choose a decision tree model

# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
#print clf  # you can inspect the learned model by printing it
clf

Training DecisionTreeClassifier...
Done!
Training time (secs): 0.015


DecisionTreeClassifier(compute_importances=None, criterion='gini',
            max_depth=None, max_features=None, max_leaf_nodes=None,
            min_density=None, min_samples_leaf=1, min_samples_split=2,
            random_state=1, splitter='best')

In [73]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print "Done!\nPrediction time (secs): {:.3f}".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')

train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 1.0


In [74]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: 1.0


In [75]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    train_start = time.time()
    train_classifier(clf, X_train, y_train)
    train_end = time.time()
    train_time = train_end - train_start
    train_F1 = predict_labels(clf, X_train, y_train)
    print "F1 score for training set: {}".format(train_F1)
    test_start = time.time()
    test_F1 = predict_labels(clf, X_test, y_test)
    print "F1 score for test set: {}".format(test_F1)
    test_end = time.time()
    test_time = test_end - test_start
    return train_time, test_time, train_F1, test_F1

# TODO: Run the helper function above for desired subsets of training data
num_train_set = [100 ,200 ,300]
#Create a dataframe to save training time, predicting time, training F1 score as well as testing F1 score
time_score_table = pd.DataFrame(columns = ['model_name','train_num','train_time',
                                           'test_time', 'train_F1', 'test_F1'])
for i, num in enumerate(num_train_set):
    table = train_predict(clf, X_train[:num],y_train[:num],X_test,y_test)
    time_score_table.loc[i,'model_name'] = clf.__class__.__name__
    time_score_table.loc[i,'train_num'] = num
    time_score_table.loc[i,['train_time', 'test_time', 'train_F1', 'test_F1']] = table
    #Display the complexity of trees
    print 'node_count:',clf.tree_.node_count
    print 'max_depth:',clf.tree_.max_depth
    print 'max_feature:',clf.max_features

# Note: Keep the test set constant

------------------------------------------
Training set size: 100
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.016
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 1.0
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: 1.0
node_count: 39
max_depth: 8
max_feature: None
------------------------------------------
Training set size: 200
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.000
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 1.0
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: 1.0
node_count: 95
max_depth: 12
max_feature: None
------------------------------------------
Training set size: 300
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.016
Predicting labels using DecisionTreeClassi

In [15]:
time_score_table

,model_name,train_num,train_time,test_time,train_F1,test_F1
0,DecisionTreeClassifier,100,0.003000021,0.0009999275,1,0.7863248
1,DecisionTreeClassifier,200,0.001999855,0.0009999275,1,0.864
2,DecisionTreeClassifier,300,0.002000093,0.001000166,1,0.9180328


The table shows that:

1. As the number of training data increases, the F1 value increases. When the number reaches 300, the F1 score of the testing dataset is over 90%.
2. It only takes a very short time to train and predict, the training time should go up as the scale of data increases, perhaps due to random error, the time does not correspond to training scale directly.
3. The F1 scores for the training data and testing data are over 90%, which means this model works very well without serious overfitting.

Also, from the node_count, max_depth and max_feature, it seems the complexity of decision tree grows in order to fit the training data. 

In [88]:
print clf.feature_importances_
print X_all.columns

[ 0.          0.          0.          0.01276552  0.0171615   0.
  0.00574449  0.01340707  0.0238018   0.00765931  0.          0.06665105
  0.03470113  0.01128008  0.0128719   0.          0.          0.00612745
  0.01021242  0.          0.0122549   0.00765931  0.01404208  0.01925153
  0.01276552  0.          0.          0.02026359  0.01735397  0.0066733
  0.01291175  0.04025101  0.10611785  0.0388254   0.01276552  0.00903599
  0.01531863  0.01021242  0.01148897  0.          0.03486298  0.02753761
  0.04191571  0.09310854  0.0159569   0.03389246  0.01097227  0.15217805]
Index([u'school_GP', u'school_MS', u'sex_F', u'sex_M', u'age', u'address_R', u'address_U', u'famsize_GT3', u'famsize_LE3', u'Pstatus_A', u'Pstatus_T', u'Medu', u'Fedu', u'Mjob_at_home', u'Mjob_health', u'Mjob_other', u'Mjob_services', u'Mjob_teacher', u'Fjob_at_home', u'Fjob_health', u'Fjob_other', u'Fjob_services', u'Fjob_teacher', u'reason_course', u'reason_home', u'reason_other', u'reason_reputation', u'guardian_fathe

It seems 'absences', 'goout' are quite significant in terms of prediction.

As this problem is a binary classification problem, and there are no missing values here, we can consider logistic regression. Logistic regression classifier can take features and multiply each one by a weight and then add them up, and put the result into the sigmoid. Anything above 0.5 will be classified as 1, otherwise as 0.

Next, we use **Logistic Regression** model which is widely used in epidemic diseases research. 
- Pros: compuatationally inexpensive, easy to implement and interpret. 
- Cons: It is prone to underfitting.

In [90]:
# TODO: Train and predict using two other models
#Choose Logistic Regression model
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=1)
# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
clf

Training LogisticRegression...
Done!
Training time (secs): 0.032


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, penalty='l2', random_state=1, tol=0.0001)

In [91]:
train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))
time_score_table = pd.DataFrame(columns = ['model_name','train_num','train_time',
                                           'test_time', 'train_F1', 'test_F1'])
for i, num in enumerate(num_train_set):
    table = train_predict(clf, X_train[:num],y_train[:num],X_test,y_test)
    time_score_table.loc[i,'model_name'] = clf.__class__.__name__
    time_score_table.loc[i,'train_num'] = num
    time_score_table.loc[i,['train_time', 'test_time', 'train_F1', 'test_F1']] = table
# Note: Keep the test set constant
time_score_table

Predicting labels using LogisticRegression...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.828828828829
Predicting labels using LogisticRegression...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.833333333333
------------------------------------------
Training set size: 100
Training LogisticRegression...
Done!
Training time (secs): 0.000
Predicting labels using LogisticRegression...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.881118881119
Predicting labels using LogisticRegression...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.884057971014
------------------------------------------
Training set size: 200
Training LogisticRegression...
Done!
Training time (secs): 0.031
Predicting labels using LogisticRegression...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.831615120275
Predicting labels using LogisticRegression...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.836879432624
------

,model_name,train_num,train_time,test_time,train_F1,test_F1
0,LogisticRegression,100,0,0.01600003,0.8811189,0.884058
1,LogisticRegression,200,0.03100014,0,0.8316151,0.8368794
2,LogisticRegression,300,0.0150001,0,0.8288288,0.8333333


The vairance of logistic regression is really small, it can prevent overfitting, but the F1 scores are lower that those of decision tree models. It takes less time to train the data, but the testing time does not look stable ,perhaps because of system random error.

Also, SVM performs well in binary classification problem. After feature transformation, all the features became numeric, it is possible for the model to create a hyperplane to separate the two classes efficiently.

Finally, we run a **Supporting Vector Machine** model which is commonly used in pattern recognition, classification and regression.
- Pros: perform well in classification problems, especially natively binary classification(like this case). It is easy ot interpret results in terms of hyperplane, and has low generalization error. It can also be applied to non-linear cases through different kernels. 
- Cons: It is sensitive to tuning parameters and kernel choice.



In [95]:
#Choose Supporting vector machine model
from sklearn import svm
clf = svm.SVC(random_state=1)
# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
clf

Training SVC...
Done!
Training time (secs): 0.047


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='rbf', max_iter=-1, probability=False, random_state=1,
  shrinking=True, tol=0.001, verbose=False)

In [96]:
train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))
time_score_table = pd.DataFrame(columns = ['model_name','train_num','train_time',
                                           'test_time', 'train_F1', 'test_F1'])
for i, num in enumerate(num_train_set):
    table = train_predict(clf, X_train[:num],y_train[:num],X_test,y_test)
    time_score_table.loc[i,'model_name'] = clf.__class__.__name__
    time_score_table.loc[i,'train_num'] = num
    time_score_table.loc[i,['train_time', 'test_time', 'train_F1', 'test_F1']] = table
# Note: Keep the test set constant
time_score_table

Predicting labels using SVC...
Done!
Prediction time (secs): 0.016
F1 score for training set: 0.874458874459
Predicting labels using SVC...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.891891891892
------------------------------------------
Training set size: 100
Training SVC...
Done!
Training time (secs): 0.000
Predicting labels using SVC...
Done!
Prediction time (secs): 0.016
F1 score for training set: 0.890322580645
Predicting labels using SVC...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.905405405405
------------------------------------------
Training set size: 200
Training SVC...
Done!
Training time (secs): 0.015
Predicting labels using SVC...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.882943143813
Predicting labels using SVC...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.910344827586
------------------------------------------
Training set size: 300
Training SVC...
Done!
Training time (secs): 0.032
Predicting

,model_name,train_num,train_time,test_time,train_F1,test_F1
0,SVC,100,0,0,0.8903226,0.9054054
1,SVC,200,0.0150001,0,0.8829431,0.9103448
2,SVC,300,0.03200006,0.01600003,0.8744589,0.8918919


The last F1 score look close to that of decision tree model, better than that in logistic regression model, and the variance is also small. But the test_F1 declines in the third test, perhaps more training data makes it difficult to find a maximal margin. And it takes more time to train and test than decision tree model. 

## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

Based on the experiments performed above, I chose decision tree as the best model, because it had good expressiveness, relatively high F1 scores than other two models on this dataset, and it was not time-consuming as SVM did. 

We can imagin a decision tree model as a tree in graph, each internal node represents a "test" on a feature, and the 'test' result decides the direction of next step. When the trained decision tree model is applied on test data, for each student the model finds a path go through the tree from the top nodes to the down leaves according to the feature values and splitting rules. For example, the root node tests whether a student is male or female, if male go to left branch, otherwise go to the right branch. Then, we move to 'age' node which tests whether a student is over 18 or not, if so go to left branch, otherwise go to right branch. Repeat the test again and again until we reach a leaf which represents a class label, and this student is classified as this label.

In [97]:
y_train_replaced = y_train.replace(['yes', 'no'], [1, 0])
y_test_replaced = y_test.replace(['yes', 'no'], [1, 0])

In [98]:
# TODO: Fine-tune your model and report the best F1 score
clf = sklearn.tree.DecisionTreeClassifier(random_state=1)
# Set up the parameters we wish to tune
parameters = {'max_features':(5,10,15,20,30,40,48)}

# Make an appropriate scoring function
from sklearn.metrics import make_scorer
scoring_function = make_scorer(f1_score,greater_is_better=True)

# Make the GridSearchCV object
from sklearn.grid_search import GridSearchCV
reg = GridSearchCV(clf, parameters, scoring_function) 
reg.fit(X_train,y_train_replaced)

GridSearchCV(cv=None,
       estimator=DecisionTreeClassifier(compute_importances=None, criterion='gini',
            max_depth=None, max_features=None, max_leaf_nodes=None,
            min_density=None, min_samples_leaf=1, min_samples_split=2,
            random_state=1, splitter='best'),
       fit_params={}, iid=True, loss_func=None, n_jobs=1,
       param_grid={'max_features': (5, 10, 15, 20, 30, 40, 48)},
       pre_dispatch='2*n_jobs', refit=True, score_func=None,
       scoring=make_scorer(f1_score), verbose=0)

In [99]:
reg.best_estimator_

DecisionTreeClassifier(compute_importances=None, criterion='gini',
            max_depth=None, max_features=40, max_leaf_nodes=None,
            min_density=None, min_samples_leaf=1, min_samples_split=2,
            random_state=1, splitter='best')

In [100]:
train_F1 = f1_score(reg.predict(X_train),y_train_replaced)
test_F1 = f1_score(reg.predict(X_test),y_test_replaced)

In [101]:
print train_F1, test_F1

1.0 1.0


I tuned the max_features parameter which started from 5 to 48(the number of features), and at the point of 30, the model got optimized with a F1 score of 0.933, better than the untuned single model. This meaned, it was not necessary to use all the features to train the model, 30 was sufficient.